In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

# 1. Embedding
embeddings = OpenAIEmbeddings()

# 2. Cache
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# 3. Vector Store
vectorstore = Chroma.from_documents(docs, embeddings)
# vector database에 질문(query)을 검색하면 질문(query)과 관련있는 문서들을 반환받게된다.
# 만약 받은 문서들이 너무 크다면 엄청 큰 문서들이 LLM에게 전달되고, 그만큼 큰 비용을 지출하게된다.
# 그래서 작은 부분으로 분할(split)해주는게 좋다. 물론 모든 문맥과 의미가 소멸할만틈 작으면 안된다.
# 처음에 캐시에 embeddings가 존재하는지 확인한다. 없다면 문서들(docs)과 함께 OpenAIEmbeddings를 사용한다.
#

In [ ]:
result = vectorstore.similarity_search("Where does winston live")
(result)

In [ ]:
#